In [1]:
import os
import copy
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io
import matplotlib.pyplot as plt
import glob
import re
from datetime import datetime
from dateutil import parser
import dateutil.parser
from scipy.io import savemat

import numpy as np
import scipy.misc
import numpy.random as rng
from PIL import Image, ImageDraw, ImageFont
from sklearn.utils import shuffle
import nibabel as nib #reading MR images
import math
from matplotlib import pyplot as plt

In [2]:
FREESURFER_A_USAR = 'freesurfer_cross_long_a_usar.csv'
ASEG_STATS = 'aseg_stats.csv'
LIPIDOMICS = 'lipidomics_v3.csv'
NAO_PROCESSADOS = 'nao_processados_a_usar_melhores_expandido.csv'
ASEG_STATS_TEST = 'aseg_stats_test.csv'
ASEG_STATS_BL_SC = 'aseg_stats_bl_sc.csv'
DATASET = 'Dataset1.0.csv'
MMSE = 'MMSE.csv'

In [ ]:
nao_processados_a_usar = pd.read_csv(NAO_PROCESSADOS)
aseg_stats = pd.read_csv(ASEG_STATS)
aseg_stats['RID'] = None
aseg_stats['VISCODE'] = None
for index, row in nao_processados_a_usar.iterrows():
    for index1, row1 in aseg_stats.iterrows():
        match = re.search(row['Image Data ID'], row1['Measure:volume'])
        if match:
            aseg_stats.at[index1, 'RID'] = row['RID']
            aseg_stats.at[index1, 'VISCODE'] = row['Visit']
            print(aseg_stats.loc[index1, 'RID'])

aseg_stats.to_csv('aseg_stats_new.csv', index=False)
        
#"S_0*([^0]+)$"

In [ ]:
freesurfer_a_usar = pd.read_csv(FREESURFER_A_USAR)
aseg_stats = pd.read_csv(ASEG_STATS)
for index, row in freesurfer_a_usar.iterrows():
    for index1, row1 in aseg_stats.iterrows():
        match1 = re.search(row['Subject'], row1['Measure:volume'])
        match2 = re.search(row['Visit'], row1['Measure:volume'])
        if match1 and match2:
            aseg_stats.at[index1, 'RID'] = row['RID']
            aseg_stats.at[index1, 'VISCODE'] = row['Visit']
            print(aseg_stats.loc[index1, 'RID'])

aseg_stats.to_csv('aseg_stats_new_new.csv', index=False)

In [31]:
aseg_stats = pd.read_csv(ASEG_STATS)
rid = aseg_stats.pop('RID')
visit = aseg_stats.pop('VISCODE')
aseg_stats.insert(0, 'RID', rid)
aseg_stats.insert(1, 'VISCODE', visit)
aseg_stats = aseg_stats.drop('Measure:volume', axis=1)
aseg_stats = aseg_stats.sort_values(by='RID')
aseg_stats.to_csv('aseg_stats_newest.csv', index=False)

In [ ]:
mrimerge = pd.read_csv('mriinfomerge.csv')
freesurfer_a_usar = pd.read_csv(FREESURFER_A_USAR)
for index, row in freesurfer_a_usar.iterrows():
    for index1, row1 in mrimerge.iterrows():
        if row['RID'] == row1['RID'] and row['VISCODE'] == row1['VISCODE']:
            mrimerge.at[index1, 'Age'] = row['Age']
            mrimerge.at[index1, 'Image Data ID'] = row['Image Data ID']
            mrimerge.at[index1, 'VISCODE'] = row['VISCODE']
            mrimerge.at[index1, 'Subject'] = row['Subject']
            mrimerge.at[index1, 'Sex'] = row['Sex']
            mrimerge.at[index1, 'Group'] = row['Group']
            mrimerge.at[index1, 'Acq Date'] = row['Acq Date']
            print(mrimerge.loc[index1, 'RID'])

mrimerge.to_csv('mrimergenew.csv', index=False)

In [18]:
lista = []
mrimerge = pd.read_csv('mrimergenew.csv')
for index, row in mrimerge.iterrows():
    if row['VISCODE'] == 'sc' or row['VISCODE'] == 'bl':
        if row['RID'] in lista:
            mrimerge = mrimerge.drop(index, axis=0)
        else:
            lista.append(row['RID'])
mrimerge = mrimerge.drop(columns=['Image Data ID', 'Subject', 'Description', 'freesurfer'])
age = mrimerge.pop('Age')
sex = mrimerge.pop('Sex')
group = mrimerge.pop('Group')
mridate = mrimerge.pop('Acq Date')
mrimerge.insert(2, 'Age', age)
mrimerge.insert(3, 'Sex', sex)
mrimerge.insert(4, 'Group', group)
mrimerge.insert(5, 'Acq Date', mridate)
mrimerge.to_csv('aseg_stats_bl_sc.csv', index=False)

In [7]:
aseg_stats_bl_sc = pd.read_csv(ASEG_STATS_BL_SC)
lipidomics = pd.read_csv(LIPIDOMICS)
aseg_stats_bl_sc.replace({'sc':'bl'}, inplace=True)
merged_df = pd.merge(aseg_stats_bl_sc, lipidomics, on=['RID', 'VISCODE'], how='inner')
merged_df = merged_df.drop_duplicates(subset=['RID', 'VISCODE'])
lipidomics_date = merged_df.pop('EXAMDATE')
viscode2 = merged_df.pop('VISCODE2')
merged_df.insert(6, 'EXAMDATE', lipidomics_date)
merged_df.insert(7, 'VISCODE2', viscode2)
merged_df.to_csv('merged.csv', index=False)

In [6]:
merged = pd.read_csv('merged.csv')
merged = merged.sort_values(by=['RID', 'Exam_Date'], ascending=[True, True])
merged.to_csv('merged_final.csv', index=False)

In [4]:
merged = pd.read_csv('aseg_stats_test.csv')
num_columns = merged.shape[1]
print(num_columns)

48


In [ ]:
dataset = pd.read_csv(DATASET)
dataset['LHC_ICV'] = None
dataset['RHC_ICV'] = None
for index, row in dataset.iterrows():
        l = round(row['Left-Hippocampus']/row['EstimatedTotalIntraCranialVol'],9)
        r = round(row['Right-Hippocampus']/row['EstimatedTotalIntraCranialVol'],9)
        dataset.at[index, 'LHC_ICV'] = l
        dataset.at[index, 'RHC_ICV'] = r
        print(dataset.loc[index, 'RID'])

dataset.to_csv('test.csv', index=False)

In [ ]:
dataset = pd.read_csv(DATASET)
mmse = pd.read_csv(MMSE)
dataset['MMSE'] = None
for index, row in dataset.iterrows():
    for index2, row2 in mmse.iterrows():
        if (row['RID'] == row2['RID']) and (row['VISCODE2'] == row2['VISCODE2']):
            dataset.at[index, 'MMSE'] = row2['MMSCORE']
            print(dataset.loc[index, 'RID'])
dataset.to_csv('Dataset3.0.csv', index=False)
